In [2]:
# Constants
# DF_PATH = "../data/processed/1_preprocessed_df.pkl"

# to work on google colabs 
url = "/content/drive/MyDrive/Colab Notebooks/related_files/1_preprocessed_df.pkl"
DF_PATH = url

NA_STRING = 'Not Specified'
TRANSPARENT_STRING = 'rgba(0, 0, 0)'

ROLE_COLS       = ['DevType']
TECH_COLS       = ['LanguageHaveWorkedWith',
                   'DatabaseHaveWorkedWith',
                   'PlatformHaveWorkedWith',
                   'WebframeHaveWorkedWith',
                   'MiscTechHaveWorkedWith',
                   'ToolsTechHaveWorkedWith',
                   'NEWCollabToolsHaveWorkedWith']
EXPORT_FEATURES_DIR = "../data/processed"

In [3]:
# mount colabs to google-drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load packages
import pandas as pd
import numpy as np
import logging
import pickle
import os

import plotly
import plotly.express as px
import plotly.graph_objects as go # Graph objects are the low-level building blocks of figures which you can use instead of Plotly Express for greater control.

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

from sklearn.pipeline import make_pipeline
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split

pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000

### Read data and preprocess data

In [5]:
# Read Data 
df = pd.read_pickle(DF_PATH)

In [6]:
# One hot encode
encoded_dfs = {}
for col in ROLE_COLS + TECH_COLS :
    binarizer = MultiLabelBinarizer()
    encoded_df = pd.DataFrame(binarizer.fit_transform(df[col]),
                              columns = binarizer.classes_,
                              index= df[col].index)
    encoded_dfs[col] = encoded_df

# Merge 1-hot encoded
ohe_df = pd.concat(encoded_dfs, axis=1)

### Dimentionality reduction

In [7]:
# Prepare sub frames
skills_ohe = ohe_df.drop('DevType', axis=1).copy()
std_skills = StandardScaler().fit_transform(skills_ohe)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [8]:
tsne_projection = TSNE(n_components=2,
                       perplexity=3,
                       learning_rate=0.01,
                       init='pca',
                       method='barnes_hut',
                       n_jobs=2,
                       n_iter=10**10,
                       random_state=0).fit_transform(std_skills)

tsne_projection = pd.DataFrame(tsne_projection, index=skills_ohe.columns)


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:986: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  FutureWarning,
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-86b4c6aa7367>", line 8, in <module>
    random_state=0).fit_transform(std_skills)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py", line 1108, in fit_transform
    embedding = self._fit(X)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py", line 1010, in _fit
    skip_num_points=skip_num_points,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py", line 1056, in _tsne
    params, kl_divergence, it = _gradient_descent(obj_func, params, **opt_args)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py", line 398, in _gradient_descent
    error, grad = objective(p, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py", line 290, in _kl_divergence

KeyboardInterrupt: ignored

In [ ]:
tsne_projection.shape